<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/01.%20Using_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-intro} -->

# Understanding LLM APIs

We will explore OpenAI models API to generate text.

<!--- @wandbcode{llmapps-intro} -->

### Setup

In [1]:
%pip install --upgrade openai==0.27.2 tiktoken wandb -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.0 MB/s eta 0:00:00


In [2]:
import os
import openai
import tiktoken
import wandb
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [14]:
from google.colab import userdata
# Recuperar a chave API do segredo salvo
openai_api_key = userdata.get('OPENAI_API_KEY')

# Definir a chave API como uma variável de ambiente
os.environ['OPENAI_API_KEY'] = openai_api_key
openai.api_key = openai_api_key


In [4]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [5]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "introduction"})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Tokenization

In [6]:
encoding = tiktoken.encoding_for_model("text-davinci-003")
enc = encoding.encode("Weights & Biases is awesome!")
print(enc)
print(encoding.decode(enc))

[1135, 2337, 1222, 8436, 1386, 318, 7427, 0]
Weights & Biases is awesome!


we can decode the tokens one by one

In [7]:
for token_id in enc:
    print(f"{token_id}\t{encoding.decode([token_id])}")

1135	We
2337	ights
1222	 &
8436	 Bi
1386	ases
318	 is
7427	 awesome
0	!


> Note how the leading tokens contain spacing.

# Sampling

Let's sample some text from the model. For this, let's create a wrapper function around the temperature parameters.
Higher temperature will result in more random samples.

In [22]:
def generate_with_temperature(temp):
  "Generate text with a given temperature, higher temperature means more randomness"
  response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    temperature=temp,
  )
  return response.choices[0].text

In [23]:
for temp in [0, 0.5, 1, 1.5, 2]:
  pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

('TEMP: 0, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and collaborate on their '
 'experiments. It offers a suite of tools for experiment tracking, '
 'hyperparameter optimization, and model visualization, making it easier for')
('TEMP: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and collaborate on their '
 'experiments. It offers a suite of tools for experiment management, '
 'hyperparameter tuning, and model visualization, making it easier for')
('TEMP: 1, GENERATION: \n'
 '\n'
 'Weights & Biases (W&B) is a machine learning platform that helps to '
 'streamline and improve the model development process. It offers tools for '
 'tracking experiments, visualizing data, and collaborating with team members. '
 'With W&B, users can easily keep')
('TEMP: 1.5, GENERATION: \

You can also use the [`top_p` parameter](https://platform.openai.com/docs/api-reference/completions/create#completions/create-top_p) to control the diversity of the generated text. This parameter controls the cumulative probability of the next token. For example, if `top_p=0.9`, the model will pick the next token from the top 90% most likely tokens. The higher the `top_p` the more likely the model will pick a token that it hasn't seen before. You should only use one of `temperature` or `top_p` at a given time.

In [26]:
def generate_with_topp(topp):
  "Generate text with a given top-p, higher top-p means more randomness"
  response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    top_p=topp,
    )
  return response.choices[0].text

In [27]:
for topp in [0.01, 0.1, 0.5, 1]:
  pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

('TOP_P: 0.01, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and collaborate on their '
 'experiments. It offers a suite of tools for experiment tracking, '
 'hyperparameter optimization, and model visualization, making it easier for')
('TOP_P: 0.1, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track, visualize, and collaborate on their '
 'experiments. It offers a suite of tools for experiment tracking, '
 'hyperparameter optimization, and model visualization, making it easier for')
('TOP_P: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is a machine learning platform that helps data scientists '
 'and researchers track, visualize, and collaborate on their machine learning '
 'experiments. It offers a variety of tools and features such as experiment '
 'tracking, hyperparameter optimization, and

# Chat API

Let's switch to chat mode and see how the model responds to our messages. We have some control over the model's response by passing a `system-role`, here we can steer to model to adhere to a certain behaviour.

> We are using `gpt-3.5-turbo`, this model is faster and cheaper than `davinci-003`

In [28]:
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say something about Weights & Biases"},
    ],
    temperature=0,
)

response

<OpenAIObject chat.completion id=chatcmpl-AVotTKwfrQfS29er7sAvnWr6G7zwn at 0x7987728a4270> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Weights & Biases is a popular machine learning experiment tracking and visualization platform that helps researchers and data scientists track and visualize their machine learning experiments. It provides tools for experiment tracking, visualization, and collaboration, making it easier to monitor and optimize machine learning models.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1732147167,
  "id": "chatcmpl-AVotTKwfrQfS29er7sAvnWr6G7zwn",
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 51,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_pre

As you can see above, the response is a JSON object with relevant information about the request.

In [29]:
pprint(response.choices[0].message.content)

('Weights & Biases is a popular machine learning experiment tracking and '
 'visualization platform that helps researchers and data scientists track and '
 'visualize their machine learning experiments. It provides tools for '
 'experiment tracking, visualization, and collaboration, making it easier to '
 'monitor and optimize machine learning models.')


In [30]:
wandb.finish()

usage/completion_tokens,▁▁▁▁▁▁▁▁▁
usage/elapsed_time,▁▁▁▁▁▁▁▁▁
usage/prompt_tokens,▁▁▁▁▁▁▁▁▁
usage/total_tokens,▁▁▁▁▁▁▁▁▁
usage/completion_tokens,50
usage/elapsed_time,0
usage/prompt_tokens,8
usage/total_tokens,58
